In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from pathlib import Path

from mc2.features.features_torch import Featurizer, MC2Loss
from mc2.training.routine import evaluate_recursively
from mc2.data_management import EXPERIMENT_LOGS_ROOT, NORMALIZATION_ROOT, load_hdf5_pretest_data
from IPython.display import display, HTML


In [ ]:
def create_multilevel_df(nested_dict):
    """Convert 3-level nested dict to DataFrame with outer keys as index and 2-level columns"""
    dfs_by_model = []
    for model_name, model_metrics in nested_dict.items():
        # Create tuples for MultiIndex columns (scenario, metric)
        tuples = [(scenario, metric) for scenario in model_metrics.keys() for metric in model_metrics[scenario].keys()]
        values = [model_metrics[scenario][metric] for scenario in model_metrics.keys() for metric in model_metrics[scenario].keys()]
        
        # Create DataFrame for this model
        df_model = pd.DataFrame([values], columns=pd.MultiIndex.from_tuples(tuples), index=[model_name])
        dfs_by_model.append(df_model)
    
    # Concatenate and set column names
    df = pd.concat(dfs_by_model, axis=0)
    #df.columns.names = ['Scenario', 'Metric']
    return df

# Test against pretest materials

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# load up model
guid_normal = {
    "3C90": "a2dc4ce1",  # best test set score was 92.78 (A/m)²
    "N87": "93c403c2",  # best test set score was 17.15 (A/m)²
}
# load up model with custom loss
guid_custom_loss = {"3C90": "09da6665", "N87": "51b88872"}
n_units = 8

In [ ]:
# helper

n_plots = 5
scenario_labels = ["10% unknown", "50% unknown", "90% unknown"]

def load_model_from_guid(mat, guid, fan_in, fan_out, device):
    mdl = torch.nn.GRU(fan_in, fan_out, batch_first=True)
    mdl.load_state_dict(torch.load(EXPERIMENT_LOGS_ROOT / f"{mat}_{guid}" / f"{mat}_{guid}.pt", map_location=device))
    mdl = mdl.to(device)
    return mdl

@torch.inference_mode()
def evaluate_pretest_scenarios(
    mat, guid, evaluate_with_gru_cell: bool, show_plots: bool = False
):
    B, T, H_init, H_true, loss, loss_short, msks_scenarios_N_tup = load_hdf5_pretest_data(mat)
    B.shape, T.shape, H_init.shape, H_true.shape
    max_H = {"3C90":  1313.42,  # A/m for 3C90 (read from training log, TODO: save this somewhere else)
         "N87":   321.14   # A/m for N87 (read from training log)
        }[mat]
    max_B = {"3C90": 0.51, "N87": 0.45}[mat]  # in T for 3C90 and N87 
    max_T = 100  # degC
    B_t = torch.tensor(B, device=device, dtype=torch.float32)
    T_t = torch.tensor(T, device=device, dtype=torch.float32)

    # prepare featurizer
    featurizer = Featurizer(mat_lbl=mat, device=device)
    # read norm constants
    norm_consts_BP = pd.read_parquet(NORMALIZATION_ROOT / f"{mat}_normalization_constants.parquet")
    featurizer.norm_consts_BP = torch.tensor(norm_consts_BP.to_numpy(), device=device, dtype=torch.float32)
    featurizer.n_inputs = featurizer.norm_consts_BP.shape[1]

    # load model
    mdl = load_model_from_guid(mat, guid, featurizer.n_inputs, n_units, device)

    H_true_t = torch.tensor(H_true, device=device, dtype=torch.float32)
    if evaluate_with_gru_cell:
        # rebuild GRU into GRUcell
        gru_cell = torch.nn.GRUCell(featurizer.n_inputs, n_units)

        def replace_key(d):
            replacement_map_d = {
                "weight_ih_l0": "weight_ih",
                "weight_hh_l0": "weight_hh",
                "bias_ih_l0": "bias_ih",
                "bias_hh_l0": "bias_hh",
            }
            return {replacement_map_d.get(k, k): v for k, v in d.items()}

        gru_cell.load_state_dict(replace_key(mdl.state_dict()))
        gru_cell = gru_cell.to(device)
        mdl_eval = gru_cell
        arch_lbl = "gru_cell"
    else:
        mdl_eval = mdl
        arch_lbl = "gru"
    H_init_t_MX = None
    print(f"Using architecture: {arch_lbl}")
    metrics_d = {}
    for scenario_i, msk_N in enumerate(msks_scenarios_N_tup):
        print(f"  Scenario {scenario_i} - {scenario_labels[scenario_i]}: ")
        if evaluate_with_gru_cell:
            H_init_t_MX = (
                torch.tensor(H_init[msk_N][: , ~np.any(np.isnan(H_init[msk_N]), axis=0)], device=device, dtype=torch.float32)
                / max_H
            )
        warm_up_len = H_init.shape[1] - np.isnan(H_init[msk_N][0]).astype(int).sum()
        B_scenario = B_t[msk_N] / max_B
        T_scenario = T_t[msk_N] / max_T
        H_true_scenario = H_true_t[msk_N] / max_H
        pretest_tensors_d = {"pretest": {"model_in_AS_l": [(B_scenario, T_scenario)], "H": [H_true_scenario]}}
        

        _, preds_MS_l = evaluate_recursively(
            mdl_eval,
            tensors_d=pretest_tensors_d,
            loss_fn=MC2Loss(),
            featurizer=featurizer,
            max_H=max_H,
            n_states=n_units,
            set_lbl="pretest",
            warm_up_phase_target_MX=H_init_t_MX,
            model_arch='gru_cell' if evaluate_with_gru_cell else 'gru',
        )
        preds = preds_MS_l[0].cpu().numpy()[:, warm_up_len:]
        H_gt_full = H_true_t.cpu().numpy()[msk_N]
        H_gt = H_gt_full[:, warm_up_len:]

        # calculate metrics
        wce_per_sequence_M = np.max(np.abs(preds - H_gt), axis=1)
        mse_per_sequence_M = np.mean((preds - H_gt) ** 2, axis=1)
        mse = np.mean(mse_per_sequence_M)
        wce = np.max(np.abs(H_gt - preds))
        sre_per_sequence = np.sqrt(mse_per_sequence_M) / np.sqrt(np.mean(H_gt**2, axis=1))  # sequence relative error
        sre_avg = np.mean(sre_per_sequence)
        sre_95th = np.percentile(sre_per_sequence, 95)
        dbdt_full = np.gradient(B[msk_N], axis=1)
        dbdt = dbdt_full[:, warm_up_len:]
        #print(f"DEBUG: {(dbdt*H_gt).sum(axis=1)=}\n{(dbdt_full*H_gt_full).sum(axis=1)=}\n{loss_short[msk_N].ravel()=}\n{loss[msk_N].ravel()=}")
        nere_per_sequence = (((dbdt * preds) - (dbdt*H_gt)).sum(axis=1))  / np.abs(loss[msk_N])# normalized energy relative error
        nere_avg = np.mean(nere_per_sequence)
        nere_95th = np.percentile(nere_per_sequence, 95)
        idx_argmax = np.argpartition(wce_per_sequence_M, -n_plots)[-n_plots:]  # worst case trajectories
        print(f"\tMSE : {mse:>7.2f} (A/m)²")
        #print(f"\tRMSE: {np.sqrt(mse):>7.2f} A/m")
        print(f"\tWCE : {wce:>7.2f} A/m")


        metrics_d[scenario_labels[scenario_i]] = {
            "mse": np.round(mse, 3).item(),
            "wce": np.round(wce, 3).item(),
            "sre_avg": np.round(sre_avg, 3).item(),
            "sre_95th": np.round(sre_95th, 3).item(),
            "nere_avg": np.round(nere_avg, 3).item(),
            "nere_95th": np.round(nere_95th, 3).item(),
        }

        # as long as the model can only take the first gt value as initialization, all scenarios
        #  will be the same. The trajectories are the same among the scenarios.

        # Differences in loss occur due to the different frame being observed for rating
        if show_plots:
            fig, axes = plt.subplots(n_plots, 1, sharex=True, sharey="col", figsize=(10, 2.5 * n_plots))
            for tst_i in range(n_plots):
                tst_idx = idx_argmax[tst_i]
                ax = axes[tst_i]
                ax.plot(H_gt[tst_idx], label="gt")
                ax.plot(preds[tst_idx], label="pred", ls="dashed")
                ax.annotate(
                    f"MSE {mse_per_sequence_M[tst_idx]:.1f} (A/m)² | WCE {wce_per_sequence_M[tst_idx]:.1f} A/m", 
                    (0.3, 0.1), xycoords=ax.transAxes
                )

            axes[0].set_title(f"Worst-case predictions for {mat} - {scenario_labels[scenario_i]}")
            axes.flatten()[0].legend()
            for ax in axes.flatten():
                ax.grid(alpha=0.3)
            for ax in axes:
                ax.set_ylabel("H in A/m")

            for ax in [axes[-1]]:
                ax.set_xlabel("Sequence step")
            fig.tight_layout()
    return metrics_d

In [ ]:
def prepare_pretest_evaluation_dict(mat):
      
    metrics_gru_cell_custom_loss = evaluate_pretest_scenarios(mat, guid_custom_loss[mat], True, show_plots=False)
    print()
    metrics_gru_cell = evaluate_pretest_scenarios(mat, guid_normal[mat], False, show_plots=False)
    hosts_d = {"3C90": {scenario_labels[0]: {"mse": None, "wce": None, "sre_avg": 0.1305, "sre_95th": 0.347, "nere_avg": 0.007623, "nere_95th": 0.01928}, # 90 % known
            scenario_labels[1]: {"mse": None, "wce": None, "sre_avg": 0.1602, "sre_95th": 0.3443, "nere_avg": 0.0341, "nere_95th": 0.05603}, # 50 % known
            scenario_labels[2]: {"mse": None, "wce": None, "sre_avg": 0.1704, "sre_95th": 0.3476, "nere_avg": 0.0618, "nere_95th": 0.07476}}, # 10 % known
            "N87": {scenario_labels[0]: {"mse": None, "wce": None, "sre_avg": 0.1962, "sre_95th": 0.521, "nere_avg": 0.007805, "nere_95th": 0.0157}, # 90 % known
            scenario_labels[1]: {"mse": None, "wce": None, "sre_avg": 0.2767, "sre_95th": 0.8498, "nere_avg": 0.02577, "nere_95th": 0.05509}, # 50 % known
            scenario_labels[2]: {"mse": None, "wce": None, "sre_avg": 0.3028, "sre_95th": 0.9999, "nere_avg": 0.04828, "nere_95th": 0.0681} # 10 % known
            }}
    all_models_d = {"hosts": hosts_d[mat], "gru_cell": metrics_gru_cell, "gru_cell_CL": metrics_gru_cell_custom_loss, }
    return all_models_d

In [ ]:
# N87
df_models_N87 = create_multilevel_df(prepare_pretest_evaluation_dict("N87"))
display(HTML(df_models_N87.T.to_html(float_format="%.3f", bold_rows=False)))

In [ ]:
# 3C90
df_models_3C90 = create_multilevel_df(prepare_pretest_evaluation_dict("3C90"))
display(HTML(df_models_3C90.T.to_html(float_format="%.3f", bold_rows=False)))

In [ ]:
display(HTML(pd.concat([df_models_N87.T, df_models_3C90.T], axis=1, keys=["N87", "3C90"]).to_html(float_format="%.3f", bold_rows=False)))

In [ ]:
import torch.nn as nn
from torch import Tensor
class HostNet(nn.Module):
    def __init__(self):
        super(HostNet, self).__init__()

        self.lstm_B = nn.LSTM(1, 6, num_layers=1, batch_first=True, bidirectional=False)

        self.lstm_H = nn.LSTM(1, 6, num_layers=1, batch_first=True, bidirectional=False)

        self.projector = nn.Sequential(
            nn.Linear(6 *2 + 2 , 6 *2 + 2),
            nn.Tanh(),
            nn.Linear(6 *2 + 2, 8),
            nn.Tanh(),
            nn.Linear(8 , 1)
        )

    def forward(self, seq_B: Tensor, seq_H: Tensor, scal: Tensor, T: Tensor, device) -> Tensor:

        seq_B = seq_B.float()
        seq_H = seq_H.float()
        scal = scal.float()
        T = T.float()

        x_B, _ = self.lstm_B(seq_B)
        x_B = x_B[:, -1, :]

        x_H, _ = self.lstm_H(seq_H)
        x_H = x_H[:, -1, :]


        output = self.projector(torch.cat((scal, T, x_B, x_H), dim=1))
        output = output.to(device)

        return output
    
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

count_parameters(HostNet())

In [ ]:
count_parameters(mdl)

In [ ]:
np.gradient(B, axis=1).shape, np.diff(B, axis=1).shape